In [1]:
from rdkit import Chem
from crest_confs_utils import (get_low_energy_conformer,
                               xtb_single_point)

In [2]:
m = Chem.MolFromSmiles('OCCO')
m = get_low_energy_conformer(m)
xtb_single_point(m, solvent='h2o')

'-15.458396912411'